In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1517953339518_0005,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.

In [7]:
%%configure -f
{ "jars": ["wasb:///jar/hwxhelloworld_2.11-1.0.jar"] }

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1517953339518_0006,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1517953339518_0006,pyspark3,idle,Link,Link,✔


In [8]:
# SQL Database & Data Warehouse Location
user = "sqladmin"
password = "Savannah!1979"
databaseHost = "bluewatersql.database.windows.net"

if not user:
  raise Exception("Please complete the Database-Settings notebook with the database login settings")

jdbc_url_db = "jdbc:sqlserver://{}:1433;database=AdventureWorksDB;user={};password={};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;".format(databaseHost, user, password)
jdbc_url_dw = "jdbc:sqlserver://{}:1433;database=AdventureWorksDW;user={};password={};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;".format(databaseHost, user, password)

None # Suppress any output

In [9]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [10]:
existingProductCategoryDF = spark.read.jdbc(jdbc_url_db, "SalesLT.ProductCategory").filter(col("ProductCategoryId") <= 100).orderBy("ProductCategoryId")

In [5]:
existingProductCategoryDF.show()

+-----------------+-----------------------+---------------+--------------------+--------------------+
|ProductCategoryID|ParentProductCategoryID|           Name|             rowguid|        ModifiedDate|
+-----------------+-----------------------+---------------+--------------------+--------------------+
|                1|                   null|          Bikes|CFBDA25C-DF71-47A...|2002-06-01 00:00:...|
|                2|                   null|     Components|C657828D-D808-4AB...|2002-06-01 00:00:...|
|                3|                   null|       Clothing|10A7C342-CA82-48D...|2002-06-01 00:00:...|
|                4|                   null|    Accessories|2BE3BE36-D9A2-4EE...|2002-06-01 00:00:...|
|                5|                      1| Mountain Bikes|2D364ADE-264A-433...|2002-06-01 00:00:...|
|                6|                      1|     Road Bikes|000310C0-BCC8-42C...|2002-06-01 00:00:...|
|                7|                      1|  Touring Bikes|02C5061D-ECDC-427...|20

In [11]:
from collections import namedtuple
from pyspark.sql.types import *

ProductCategory = namedtuple("ProductCategory", [
  "ProductCategoryID",
  "ParentProductCategoryID",
  "Name",
])

schema=StructType([
  StructField("ProductCategoryID", IntegerType()),
  StructField("ParentProductCategoryID", IntegerType()),
  StructField("Name", StringType()),
])
newProductCategoryDF = spark.createDataFrame([
    (101, None, "Computers"),
    (102, 101,  "Laptop"),
    (103, 101,  "Tablet"),
  ], schema)
newProductCategoryDF.show()

+-----------------+-----------------------+---------+
|ProductCategoryID|ParentProductCategoryID|     Name|
+-----------------+-----------------------+---------+
|              101|                   null|Computers|
|              102|                    101|   Laptop|
|              103|                    101|   Tablet|
+-----------------+-----------------------+---------+

In [12]:
(newProductCategoryDF.repartition(1).write
  .format("main.scala")
  .mode("append")
  .option("url", jdbc_url_db)
  .option("dbtable", "SalesLT.ProductCategory")
  .save())

An error occurred while calling o97.save.
: java.lang.NoSuchMethodError: org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.tableExists(Ljava/sql/Connection;Lorg/apache/spark/sql/execution/datasources/jdbc/JDBCOptions;)Z
	at main.scala.DefaultSource.createRelation(DefaultSource.scala:43)
	at org.apache.spark.sql.execution.datasources.DataSource.write(DataSource.scala:518)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:215)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:280)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at 